## Dijkstra method python code
### by group 13

### User Manual
1.put .csv file and the code in the same file.

2.run the whole block.

3.input the csv file name, note that csv file must be coded by UTF-8(without BOM).

4.input the start node, note that the node will start from 1.

5.check whether you want to save the result as a Microsoft Excel File, type "Y" to save the result, if not, type anything else.

### Note
1.The Dijkstra method is written in function "dijkstra_method()" which input the cost-matrix & a selected start point, output both distance & the path information in dict form.

2.Only accept matrix form conservative graphs, or the program will return warning messages. 

3.The program will automatically remove all loops and return warning messages. 

4.The program accept unsymmetric cost-matrix and will return message when there’s no available path. 


In [3]:
# coding in utf-8
# Author: Chuxi Zhu
import csv
import pandas as pd
"""
read csv file
"""
def read_data(filename:str):
    data = csv.reader(open(filename,"r"))
    cost_matrix = []
    for line in data:
        cost_matrix.append(line)
    return cost_matrix

"""
transform cost matrix with empty data,also convert string type data into float type data.
"""
def formalized_matrix(cost_matrix):
    invalid = False # Check whether there's a negative distance
    for i in range(len(cost_matrix)):
        if len(cost_matrix[i]) != len(cost_matrix):
            print("Error: there is a problem on matrix shape")
        else:
            for j in range(len(cost_matrix)):
                if i == j:
                    if cost_matrix[i][j] != "0":
                        print("Warning: a loop has been removed!")
                    else:
                        cost_matrix[i][j]=0
                else:
                    if cost_matrix[i][j] == '' or cost_matrix[i][j] == 10000:
                        cost_matrix[i][j]=float('inf')
                    else:
                        cost_matrix[i][j] = float(cost_matrix[i][j])
                        if float(cost_matrix[i][j]) <= 0:
                            invalid = True
                        else:
                            pass
    return cost_matrix, invalid

"""
perform Dijkstra method, input cost_matrix and the start point, output distance and paths.
"""
def dijkstra_method(cost_matrix, start):
    start:int = start # get the start point
    inf = float('inf')
    
    # set up and initialized temp variables in this function
    vertex_num = len(cost_matrix) # number of vertex
    visited = [0] * vertex_num # visted vertex would be put here
    dis = {vertex: cost_matrix[start][vertex] for vertex in range(vertex_num)} # get distance from start vertex  
    parents = {vertex: -1 for vertex in range(vertex_num)} # if -1 means the vertex itself
    visited[start] = 1 # put the start vertex in first iteration
    last_point = start # the vertex that will be set visited next, initialized as start vertex
    
    for i in range(vertex_num - 1): # iterate no more than n-1 times
        if visited == [1] * vertex_num:
            pass
        else:
            min_distance = inf # min in this iteration
            for j in range(vertex_num): # find min of the adjacent distance
                if visited[j] == 0 and dis[j] < min_distance:
                    min_distance = dis[j]
                    last_point = j
                    visited[last_point] = 1
                for k in range(vertex_num): # renew parents and distance by new vertex
                    if cost_matrix[last_point][k] < inf and dis[k] > dis[last_point] + cost_matrix[last_point][k]:
                        dis[k] = dis[last_point] + cost_matrix[last_point][k]
                        parents[k] = last_point # update last point
            
    return {key: values for key, values in dis.items()}, {key: values for key, values in parents.items()}


"""
Transform the result into DataFrame, make the output easier for reading or transforming to Excel file.
"""
def form_result(dis:dict):
    output = pd.DataFrame(dis,columns=["end_point","shortest distance"])
    for i in range(len(dis)):
        temp = {"end_point":int(i+1),"shortest distance":dis[i]}
        output = output.append(temp,ignore_index=True)
    output.set_index("end_point",inplace=True,drop=True)
    return(output)

"""
the original path out put is hard to read, this function makes the path easier for reading or transforming to Excel file
"""
def show_path(start_point,dis:dict,parents:dict):
    start_point += 1 
    pathput = pd.DataFrame(dis,columns=["end_point","path"]) # The result DataFrame
    path_node = [x for x in range(1, len(parents)+1)]      
    for i in range(len(parents)):
        if dis[i] != inf:
            temp_path = [] # to record the path node
            temp_path.append(path_node[i]) # add sink to the path node
            if parents[i] == -1:
                temp_path.insert(0,start_point) # the sink is directly connected with source
            else:
                adding = parents[i]
                while adding != -1 and adding != start_point:
                    temp_path.insert(0,adding+1) # add path point
                    adding = parents[adding]
                temp_path.insert(0,start_point) # add start point
            pathput = pathput.append({"end_point":path_node[i],"path":temp_path},ignore_index=True) # form an dict
        else:
            pathput = pathput.append({"end_point":path_node[i],"path":None},ignore_index=True)
    pathput.set_index("end_point",inplace=True,drop=True) # add dict to the result DataFrame
    return(pathput)

"""
save the result in a Microsoft Excel file
"""
def save_result(output,pathput):
    with pd.ExcelWriter("Dijkstra_result.xlsx")as writer:
        output.to_excel(writer,sheet_name="shortest_distance")
        pathput.to_excel(writer,sheet_name="shortest_path_node")
        print("result saved as Dijkstra_method.xlsx")

if __name__ == '__main__':
    inf = float('inf')
    # return friendly message when program falls
    try:
        cost_matrix = read_data(input("please input the name, for example abc.csv"))
    except FileNotFoundError:
        print("Error: File not found.")
    except UnicodeDecodeError:
        print("Error: Please use the csv file coded by UTF-8")
    else:
        cost_matrix,invalid = formalized_matrix(cost_matrix)
        if invalid:
            print("Error: negative distance exists.")
        else:
            try:
                start_point = int(input("input the start node, the first node starts from 1:"))-1
                choice = str(input("if you want to save the the result as excel, please type Y "))
            except ValueError:
                print("ValueError: please input a valid interger")
            else:
                if start_point >= len(cost_matrix) or start_point < 0:
                    print("ValueError: please input right start node")
                else:
                    dis, parents = dijkstra_method(cost_matrix, start_point)
                    output = form_result(dis)
                    pathput = show_path(start_point,dis,parents)
                    print(output)
                    print("Note: an inf result means it's impossile to reach the point")
                    print(pathput)
                    print("Note: NaN means no available path")
                    if choice == "Y":
                        save_result(output,pathput)

please input the name, for example abc.csvchina_road.csv
input the start node, the first node starts from 1:5
if you want to save the the result as excel, please type Y 
           shortest distance
end_point                   
1.0                   1468.5
2.0                    938.0
3.0                    426.5
4.0                    215.0
5.0                      0.0
6.0                   1341.0
7.0                   1427.0
8.0                   1100.0
9.0                    962.5
10.0                   647.0
11.0                   990.0
12.0                  2316.0
13.0                  1584.0
14.0                  1054.5
Note: an inf result means it's impossile to reach the point
                        path
end_point                   
1              [5, 3, 14, 1]
2                  [5, 4, 2]
3                     [5, 3]
4                     [5, 4]
5                     [5, 5]
6                 [5, 10, 6]
7                 [5, 10, 7]
8                  [5, 3, 8]
9               